In [ ]:
import yt
import pythreejs
import yt.utilities.lib.mesh_triangulation as mt

In [ ]:
ds = yt.load_sample("MOOSE_sample_data/mps_out.e", step=-1)
nt = ds.parameters['_dt'].size

In [ ]:
for i in range(nt):
    ds = yt.load_sample("MOOSE_sample_data/mps_out.e", step=i, displacements = {'connect1': (1.0, [0.0, 0.0, 0.0]), 'connect2': (1.0, [0.0, 0.0, 0.0])})
    for ax in 'xyz':
        s = yt.SlicePlot(ds, ax, ("all", "hydrostatic_stress"))
        s.annotate_mesh_lines(color='k')
        s.save(f"frames/moose_hydrostatic_{ax}_{i:03d}")

In [ ]:
ds = yt.load_sample("SecondOrderTets", step=-1, displacements = {'connect1': ([1.0, 1.0, 1.0], [0.0,0.0,0.0])})

In [ ]:
ds.index.meshes[0]._index_offset

In [ ]:
s = yt.SlicePlot(ds, "y", ("all", "ux"))
s.annotate_mesh_lines(color = 'k')
s.show()

In [ ]:
cam = pythreejs.PerspectiveCamera(
    position=[25, 35, 100], fov=20, children=[pythreejs.AmbientLight()],
)
children = [cam, pythreejs.AmbientLight(color="#dddddd")]
material = pythreejs.MeshBasicMaterial(
    color="#ff0000", vertexColors="VertexColors", side="DoubleSide"
)

def mesh_to_geometry(mesh):
    indices = mt.triangulate_indices(mesh.connectivity_indices - mesh._index_offset)
    # We need to convert these to the triangulated mesh, which is done inside mesh_triangulation.pyx
    attributes = dict(
        position=pythreejs.BufferAttribute(mesh.connectivity_coords, normalized=False),
        index=pythreejs.BufferAttribute(
           indices[:,(0,1,2)].ravel(order="C").astype("u4"), normalized=False
        ),
        color = pythreejs.BufferAttribute(
            (mesh.connectivity_coords * 255).astype('u1')
        )
    )
    geometry = pythreejs.BufferGeometry(attributes=attributes)
    geometry.exec_three_obj_method("computeFaceNormals")
    return geometry

for mesh in ds.index.meshes:
    geometry = mesh_to_geometry(mesh)
    children.append(pythreejs.Mesh(
        geometry=geometry, material=material, position=[0, 0, 0]
    ))

scene = pythreejs.Scene(children=children)

rendererCube = pythreejs.Renderer(
    camera=cam,
    background="white",
    background_opacity=1,
    scene=scene,
    controls=[pythreejs.OrbitControls(controlling=cam)],
    width=800,
    height=800,
)


rendererCube